In [1]:
import numpy as np
from math import sqrt
import random




class Player:
    def __init__(self, name, settlements, roads):
        self.name=name
        self.resources ={'brick': 0, 'grain':0, 'lumber': 0, 'ore':0, 'wool':0} #Resources (initialized as 0) 
        self.points=0
        self.settlements=[]
        self.roads=[]
        self.image_roads=[]
        self.image_settlement=None
        self.images_city=None
        self.available_settlements=[s for s in settlements]
        self.available_roads=[r for r in roads]



    def buy_road(self, road, round):
        road.owner=self.name
        self.roads.append(road)
        if round==1:
            self.resources['brick']-=1
            self.resources['lumber']-=1


    def buy_settlement(self,  settlement, round):
        settlement.owner=self.name
        settlement.multiplier=1
        self.settlements.append(settlement)
        self.points+=1
        if round==1:
            self.resources['brick']-=1
            self.resources['lumber']-=1
            self.resources['wool']-=1
            self.resources['grain']-=1

    def upgrade_settlement(self, settlement):
        settlement.upgrade()
        self.points+=1
        self.resources['ore']-=3
        self.resources['grain']-=2







class Robber:
    def __init__(self):
        self.region=None



class Region:

    types_to_resources={'4': 'brick', '3':'grain', '1':'lumber', '5':'ore', '2':'wool', '6':'desert'}  #forest=1 (lumber), pasture=2 (wool), fields=3 (grain), hills=4 (brick), mountain=5 (ore), desert=6 (None)
    

    def __init__(self, id, dice_number, region_type, coordinates):
        self.coordinates=coordinates
        self.region_type=region_type
        self.resource_type=self.types_to_resources[str(region_type)]
        self.id=id
        self.dice_number=dice_number
        self.robber=0
        self.vertices={p:  None for p in ['v1', 'v2', 'v3', 'v4', 'v5', 'v6']}
        self.edges={p: None for p in ['e1', 'e2', 'e3', 'e4', 'e5', 'e6']}

    




class Road:

    def __init__(self,coordinates, *args):
        self.owner=None
        self.regions=[name for name in args]
        self.connected_roads=[]
        self.connected_settlements=[]
        self.coordinates=coordinates





class Settlement:

    #each player has X settlements and Y roads at the beggining
    def __init__(self,coordinates, *args):
        self.multiplier=0 
        self.owner=None
        self.regions=[name for name in args]
        self.connected_roads=[]
        self.connected_settlements=[]
        self.coordinates=coordinates


    def upgrade(self):
        self.multiplier=2






    



class Board:

    def __init__(self):
        
        self.region_position=tuple(range(0, 19))
        self.region_types=[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5]#desert needs to have 7 
        random.shuffle(self.region_types)
        self.region_types.append(6)
        #forest=1 (lumber), pasture=2 (wool), fields=3 (grain), hills=4 (brick), mountain=5 (ore), desert=6 (None)

        self.region_coordinates={'1': (-1,-sqrt(3)), '2': (0,-sqrt(3)), '3':(1,-sqrt(3)), '4':(1.5,-sqrt(3/4)), '5': (2,0), '6':(1.5, sqrt(3/4)), '7':(1,sqrt(3)), '8':(0,sqrt(3)), '9':(-1,sqrt(3)), 
                                '10': (-1.5,sqrt(3/4)), '11': (-2,0), '12': (-1.5,-sqrt(3/4)), '13':(-0.5,-sqrt(3/4)), '14':(0.5,-sqrt(3/4)), '15': (1,0), '16':(0.5,sqrt(3/4)), '17':(-0.5,sqrt(3/4)), 
                                '18':(-1,0), '19':(0,0)} #change

        self.region_numbering=[('a',5) , ('b',2) , ('c',6), ('d',3), ('e',8), ('f',10), ('g',9), ('h',12), ('i',11), 
                            ('j',4), ('k',8), ('l',10), ('m',9), ('n',4), ('o',5), ('p',6), ('q',3), ('r',11), ('desert',7)]
        self.region_parameter=[(x,y) for x,y in zip(self.region_types,self.region_numbering)]
        random.shuffle(self.region_parameter)
        self.regions=[]
        self.roads=[]
        self.settlements=[]
        self.proximity_vector={(-0.5,-sqrt(3/4)):(('v4', 'v5', 'e4'), ('v2', 'v1', 'e1')), 
                                (0.5,-sqrt(3/4)):(('v4', 'v3', 'e3'), ('v6', 'v1', 'e6')), 
                                (1,0):(('v2', 'v3', 'e2'), ('v6', 'v5', 'e5')),
                                (0.5,sqrt(3/4)):(('v2', 'v1', 'e1'),('v4', 'v5', 'e4')), 
                                (-0.5, sqrt(3/4)):(('v6', 'v1', 'e6'),('v4', 'v3', 'e3')), 
                                (-1,0):(('v6', 'v5', 'e5'),('v2', 'v3', 'e2'))} 
        self.edge_coordinate_mapping={'e1':(0.25, sqrt(3/4)/2),'e2':(1/2, 0), 'e3':(0.25, -sqrt(3/4)/2), 'e4':(-0.25, -sqrt(3/4)/2), 'e5':(-1/2, -0), 'e6':(-0.25, sqrt(3/4)/2)}
        self.vertex_coordinate_mapping={'v1':(0, 1/sqrt(3)),'v2':(1/2, 1/(2*sqrt(3))), 'v3':(1/2, -1/(2*sqrt(3))), 'v4':(0, -1/sqrt(3)), 'v5':(-1/2, -1/(2*sqrt(3))), 'v6':(-1/2, 1/(2*sqrt(3)))}
        self.connection_vertices_vertices={'v1':('v6', 'v2'),'v2':('v1', 'v3'),'v3':('v2', 'v4'),
                                            'v4':('v3', 'v5'),'v5':('v4', 'v6'),'v6':('v5', 'v1')}
        self.connection_vertices_edges={'v1':('e6', 'e1'),'v2':('e1', 'e2'),'v3':('e2', 'e3'),
                                            'v4':('e3', 'e4'),'v5':('e4', 'e5'),'v6':('e5', 'e6')}
        self.connection_edges_vertices={'e1':('v1', 'v2'),'e2':('v2', 'v3'),'e3':('v3', 'v4'),
                                            'e4':('v4', 'v5'),'e5':('v5', 'v6'),'e6':('v6', 'v1')}
        self.connection_edges_edges={'e1':('e6', 'e2'),'e2':('e1', 'e3'),'e3':('e2', 'e4'),
                                            'e4':('e3', 'e5'),'e5':('e4', 'e6'),'e6':('e5', 'e1')}
        

        
    def setup_roads_settlements(self, region1, region2):
        self.vector=region2.coordinates[0]-region1.coordinates[0], region2.coordinates[1]-region1.coordinates[1]
        if self.vector in self.proximity_vector:
            if region1.edges[self.proximity_vector[self.vector][0][2]]==None:
                if region2.edges[self.proximity_vector[self.vector][1][2]]==None:
                    region1.edges[self.proximity_vector[self.vector][0][2]]=Road(tuple(np.array(region1.coordinates)+np.array(self.edge_coordinate_mapping[self.proximity_vector[self.vector][0][2]])),region1, region2 )
                    region2.edges[self.proximity_vector[self.vector][1][2]]=region1.edges[self.proximity_vector[self.vector][0][2]] #adding the already created road to the other region
                    self.roads.append(region1.edges[self.proximity_vector[self.vector][0][2]]) #adding the newly created road to the list of 
                else:
                    region2.edges[self.proximity_vector[self.vector][1][2]].regions.append(region1) #appending a region id to road.region which is a list containing all regions sharing that edge
                    region1.edges[self.proximity_vector[self.vector][0][2]]=region2.edges[self.proximity_vector[self.vector][1][2]]
        

            if region1.vertices[self.proximity_vector[self.vector][0][0]]==None:
                if region2.vertices[self.proximity_vector[self.vector][1][0]]==None:
                    region1.vertices[self.proximity_vector[self.vector][0][0]]=Settlement(tuple(np.array(region1.coordinates)+np.array(self.vertex_coordinate_mapping[self.proximity_vector[self.vector][0][0]])),region1, region2)
                    region2.vertices[self.proximity_vector[self.vector][1][0]]=region1.vertices[self.proximity_vector[self.vector][0][0]]
                    self.settlements.append(region1.vertices[self.proximity_vector[self.vector][0][0]]) #adding the newly created settlement to the list of settlements
                else:
                    region2.vertices[self.proximity_vector[self.vector][1][0]].regions.append(region1)
                    region1.vertices[self.proximity_vector[self.vector][0][0]]=region2.vertices[self.proximity_vector[self.vector][1][0]]
            else:
                region1.vertices[self.proximity_vector[self.vector][0][0]].regions.append(region2)
                region2.vertices[self.proximity_vector[self.vector][1][0]]=region1.vertices[self.proximity_vector[self.vector][0][0]]


            if region1.vertices[self.proximity_vector[self.vector][0][1]]==None:
                if region2.vertices[self.proximity_vector[self.vector][1][1]]==None:
                    region1.vertices[self.proximity_vector[self.vector][0][1]]=Settlement(tuple(np.array(region1.coordinates)+np.array(self.vertex_coordinate_mapping[self.proximity_vector[self.vector][0][1]])),region1, region2)
                    region2.vertices[self.proximity_vector[self.vector][1][1]]=region1.vertices[self.proximity_vector[self.vector][0][1]]
                    self.settlements.append(region1.vertices[self.proximity_vector[self.vector][0][1]]) #adding the newly created settlement to the list of settlements
                else:
                    region2.vertices[self.proximity_vector[self.vector][1][1]].regions.append(region1)
                    region1.vertices[self.proximity_vector[self.vector][1][0]]=region2.vertices[self.proximity_vector[self.vector][1][1]]
            else:
                region1.vertices[self.proximity_vector[self.vector][0][1]].regions.append(region2)
                region2.vertices[self.proximity_vector[self.vector][1][1]]=region1.vertices[self.proximity_vector[self.vector][0][1]]


    def connections(self): 
        for region in self.regions:
            for v in region.vertices:
                for k in [0,1]:
                    if region.vertices[self.connection_vertices_vertices[v][k]] not in region.vertices[v].connected_settlements:
                        region.vertices[v].connected_settlements.append(region.vertices[self.connection_vertices_vertices[v][k]])
                    if region.edges[self.connection_vertices_edges[v][k]] not in region.vertices[v].connected_roads:
                        region.vertices[v].connected_roads.append(region.edges[self.connection_vertices_edges[v][k]])

            for v in region.edges:
                for k in [0,1]:
                    if region.edges[self.connection_edges_edges[v][k]] not in region.edges[v].connected_roads:
                        region.edges[v].connected_roads.append(region.edges[self.connection_edges_edges[v][k]])
                    if region.vertices[self.connection_edges_vertices[v][k]] not in region.edges[v].connected_settlements:
                        region.edges[v].connected_settlements.append(region.vertices[self.connection_edges_vertices[v][k]])  
                        
                    
                    



    def setup_board(self):

        for k in range(0,19):
            self.regions.append(Region(self.region_parameter[k][1][0],self.region_parameter[k][1][1],self.region_parameter[k][0], self.region_coordinates[str(k+1)]))

        for i in range(len(self.regions)): #creating all inner roads and settlements and assigning them to the relative regions
            for j in range(len(self.regions)):
                self.setup_roads_settlements(self.regions[i],self.regions[j])

                # create outer roads and settlements and assign them to the respective region

        self.k=0
        for p in self.regions:
            for q in self.edge_coordinate_mapping:
                if p.edges[q]==None:
                    p.edges[q]=Road(tuple(np.array(p.coordinates)+np.array(self.edge_coordinate_mapping[q])),p)
                    self.roads.append(p.edges[q]) #adding the newly created road to the list of roads      
            for q in self.vertex_coordinate_mapping:
                if p.vertices[q]==None:
                    self.k+=1
                    p.vertices[q]=Settlement(tuple(np.array(p.coordinates)+np.array(self.vertex_coordinate_mapping[q])),p) 
                    self.settlements.append(p.vertices[q])#adding the newly created settlement to the list of settlements
        self.connections()
        self.robber=Robber()
        for x in self.regions:
            if x.dice_number == 7:
                self.robber.region=x
                x.robber=1

    
    def move_robber(self, region):
        self.robber.region.robber=0
        self.robber.region=region
        region.robber=1
        

    def round0_available_settlements(self, player):
        player.available_settlements = self.settlements.copy()
        self.settlements_to_remove=[]
        for v in player.available_settlements: #v stands for vertex
            if v.owner != None:
                self.settlements_to_remove.append(v)
        for v in self.settlements_to_remove:
             for cv in v.connected_settlements:
                 if cv in player.available_settlements:
                     player.available_settlements.remove(cv)

        for s in self.settlements_to_remove:
            player.available_settlements.remove(s)

                   


    def round0_available_roads (self, player):
        player.available_roads = self.roads.copy()
        self.roads_to_remove=[]
        for r in player.available_roads: #r stands for road
            if (player.name not in [cs.owner for cs in r.connected_settlements]) and (player.name not in [cr.owner for cr in r.connected_roads]):
                self.roads_to_remove.append(r)
            if r.owner!=None and (r not in self.roads_to_remove):
                self.roads_to_remove.append(r)
        for r in self.roads_to_remove:
            player.available_roads.remove(r)
            



    def round_available_settlements (self, player):
        player.available_settlements = self.settlements.copy()
        self.settlements_to_remove=[]
        for v in player.available_settlements: #v stands for vertex
            if v.owner != None:
                self.settlements_to_remove.append(v)
        for v in self.settlements_to_remove:
             for cv in v.connected_settlements:
                 if cv in player.available_settlements:
                     player.available_settlements.remove(cv) 

        for s in self.settlements_to_remove:
            player.available_settlements.remove(s)

        self.settlements_to_remove=[]
        for s in  player.available_settlements:
            if player.name not in [cr.owner for cr in s.connected_roads]: 
                self.settlements_to_remove.append(s)
        
        for s in self.settlements_to_remove:
            player.available_settlements.remove(s)
        


    def round_available_roads (self, player):
        player.available_roads = self.roads.copy()
        self.roads_to_remove=[]
        for r in player.available_roads: #r stands for road
            if (player.name not in [cs.owner for cs in r.connected_settlements]) and (player.name not in [cr.owner for cr in r.connected_roads]):
                self.roads_to_remove.append(r)
            if r.owner!=None and (r not in self.roads_to_remove):
                self.roads_to_remove.append(r)
        for r in self.roads_to_remove:
            player.available_roads.remove(r)







class Game:

    def __init__(self, board, GUI):
        self.board=board
        self.players={}
        for name in ['player1', 'player2', 'player3', 'player4']:
            self.players[name]=Player(name, self.board.settlements, self.board.roads)
            self.scores=[self.players[player].points for player in self.players]
            self.winning_condition=0
        self.current_player=None
        self.GUI=GUI


    #fix indentation
    def declare_winner(self, current_player):
        if self.players[current_player].points>5: 
            return 1
        else:
            return 0

    def start_game(self):
        self.round_zero()
        self.rounds()
        self.GUI.display_winner(self.current_player)

    def round_zero(self):
        for player in self.players:
            self.turn_zero(player)
        for player in [player for player in self.players][::-1]:
            self.turn_zero(player)
        self.give_resources_round0()



        
    def buy_settlement(self):
        self.GUI.show_buttons_settlements(round=0)
        
    def buy_road(self):
        self.GUI.show_buttons_roads(round=0)


    def rounds(self):
        while self.winning_condition==0:
            for player in self.players:
                self.turn(player)
                self.winning_condition=self.declare_winner(player)
                if self.winning_condition!=0:
                    return None
        



    def turn_zero(self, player):
        self.current_player=player
        self.GUI.update_player_label()
        self.buy_settlement()
        self.buy_road()


    def turn(self, player):
        self.current_player=player
        self.GUI.update_player_label()
        self.dice_result= random.randint(1,6)+random.randint(1,6)
        self.GUI.update_dice_roll(self.dice_result)
        self.give_resources(self.dice_result)
        self.GUI.update_player_resources()
        if self.dice_result==7:
            self.GUI.action_place_robber()
        self.GUI.other_actions()

    def give_resources_round0(self):
        for player in self.players:
            self.get_resources_round0(self.players[player].settlements[1])

    def give_resources(self, dice):
        if dice==7:
            pass
        for player in self.players:
            for s in self.players[player].settlements:
                self.get_resources(s, dice)

    def get_resources_round0(self, settlement):
        for region in set(settlement.regions):
            if region.resource_type=='desert':
                continue
            self.players[settlement.owner].resources[region.resource_type]+=1*settlement.multiplier

    def get_resources(self, settlement, dice_result):
        for region in set(settlement.regions):
            if region.resource_type=='desert':
                continue
            if region.robber==1:
                continue
            if region.dice_number==dice_result:
                self.players[settlement.owner].resources[region.resource_type]+=1*settlement.multiplier
        



In [2]:
import tkinter as tk
from tkinter import * 
from tkinter import ttk
from PIL import Image, ImageTk


In [3]:
class GUI:
    def __init__(self):
        self.board=Board()
        self.master = tk.Tk()
        self.game=Game(self.board, self)
        self.var=tk.IntVar()
        self.var_end_turn=tk.IntVar()
        self.round=0
        
        self.frame = tk.LabelFrame(self.master, text = "Coords Here", relief = tk.FLAT)
        self.frame.grid(sticky = tk.NSEW)

        self.map = tk.Canvas(
            self.frame, 
            highlightthickness = 0, background = "dodgerblue",
            width = 1000, height = 1000, takefocus = 1,
            scrollregion = "-500 -500 500 500")
        self.map.grid(sticky = tk.NSEW, row=0, column=0)


        self.panel = tk.Canvas(
            self.frame, 
            highlightthickness = 0, background = "wheat",
            width = 500, height = 1000, takefocus = 1)
        self.panel.grid(sticky = tk.NSEW, row=0, column=1)

        self.rescale=195

        self.button_robber=dict()
        self.button_settlement=dict()
        self.button_city=dict()
        self.button_road=dict()


        self.robber = PhotoImage(file="PNGs/PNGs/Additional_elements/robber.png")

        
        
        self.start_button= Button(self.panel, text='START GAME', command=self.start_game, bg="green")
        self.start_button.place(in_=self.panel,x=200,y=450, height=100, width=100 )


        self.robber = PhotoImage(file="PNGs/PNGs/Additional_elements/robber.png")
        for player in self.game.players:
            self.game.players[player].image_roads.append(PhotoImage(file=f"PNGs/PNGs/builds/{ self.game.players[player].name}/road1.png"))
            self.game.players[player].image_roads.append(PhotoImage(file=f"PNGs/PNGs/builds/{ self.game.players[player].name}/road2.png"))
            self.game.players[player].image_roads.append(PhotoImage(file=f"PNGs/PNGs/builds/{ self.game.players[player].name}/road3.png"))
            self.game.players[player].image_city = PhotoImage(file=f"PNGs/PNGs/builds/{ self.game.players[player].name}/city.png")
            self.game.players[player].image_settlement=PhotoImage(file=f"PNGs/PNGs/builds/{ self.game.players[player].name}/colony.png")


        self.road_rotation={(0.25, round(sqrt(3/4)/2,10)): 1, (1/2, 0): 0, (0.25, round(-sqrt(3/4)/2,10)): 2, (-0.25, round(-sqrt(3/4)/2,10)): 1, (-1/2, -0): 0, (-0.25, round(sqrt(3/4)/2,10)): 2}
        
        self.RGB_players={'player1': 'green3', 'player2': 'red', 'player3': 'MediumOrchid2', 'player4': 'DodgerBlue2'}


    def start_game(self):
        self.start_button.destroy()
        self.board.setup_board()

        self.resource_type = dict()
        self.number=dict()
        for region in self.board.regions:

            self.resource_type[region]=ImageTk.PhotoImage(Image.open(f"PNGs/PNGs/Resources/{region.resource_type}.png").resize((200,226)))
            self.number[region] = ImageTk.PhotoImage(Image.open(f"PNGs/PNGs/Numbers/{region.dice_number}.png").resize((220,249)))
        self.setup_map()
        self.setup_panel()
        self.game.start_game()



        
 
    def move_robber(self,region):
        self.map.move(self.robber_image, (region.coordinates[0]-self.board.robber.region.coordinates[0])*self.rescale, (region.coordinates[1]-self.board.robber.region.coordinates[1])*self.rescale)
        self.board.move_robber(region)
        for x in self.button_robber.values():
            x.state(['disabled'])
            x.place_forget()
        self.enable_button(self.end_action_button)



    def place_settlement(self,settlement):
        self.settlement_image=self.game.players[self.game.current_player].image_settlement
        self.game.players[self.game.current_player].buy_settlement(settlement, self.round) #replace with buy settlement method from player
        self.update_player_resources()
        self.map.create_image(settlement.coordinates[0]*self.rescale, settlement.coordinates[1]*self.rescale, image = self.settlement_image)
        for x in self.button_settlement.values():
            x.place_forget()
        self.enable_button(self.end_action_button)

    def place_city(self,settlement):
        self.city_image=self.game.players[self.game.current_player].image_city
        self.game.players[self.game.current_player].upgrade_settlement(settlement) #replace with buy settlement method from player
        self.update_player_resources()
        self.map.create_image(settlement.coordinates[0]*self.rescale, settlement.coordinates[1]*self.rescale, image = self.city_image)
        for x in self.button_city.values():
            x.place_forget()
        self.enable_button(self.end_action_button)


    def place_road(self, road):
        self.vector=(round(road.coordinates[0]-road.regions[0].coordinates[0],10), round(road.coordinates[1]-road.regions[0].coordinates[1],10) )
        self.game.players[self.game.current_player].buy_road(road, self.round) #replace with buy settlement method from player
        self.update_player_resources()
        self.road_image=self.game.players[self.game.current_player].image_roads[self.road_rotation[self.vector]]
        self.map.create_image(road.coordinates[0]*self.rescale, road.coordinates[1]*self.rescale, image = self.road_image)
        for x in self.button_road.values():
            x.place_forget()
        self.enable_button(self.end_action_button)      



    def place_image_region(self, region):
        self.map.create_image(region.coordinates[0]*self.rescale, region.coordinates[1]*self.rescale, image = self.resource_type[region])
        self.map.create_image(region.coordinates[0]*self.rescale, region.coordinates[1]*self.rescale, image = self.number[region])
        self.button_robber[region]= ttk.Button(self.map, text='move here',command=lambda : self.move_robber(region))
        self.button_robber[region].place(in_=self.map, x=region.coordinates[0]*self.rescale+480, y=region.coordinates[1]*self.rescale+480)
        self.button_robber[region].place_forget()
    


    def create_button_vertices(self, settlement, city):
        if city==0:
            self.button_settlement[settlement]=ttk.Button(self.map ,text='set',command=lambda : self.place_settlement(settlement))
            self.button_settlement[settlement].place(in_=self.map, x=settlement.coordinates[0]*self.rescale+480, y=settlement.coordinates[1]*self.rescale+480)
            self.button_settlement[settlement].place_forget()
        if city==1:
            self.button_city[settlement]=ttk.Button(self.map ,text='city',command=lambda : self.place_city(settlement))
            self.button_city[settlement].place(in_=self.map, x=settlement.coordinates[0]*self.rescale+480, y=settlement.coordinates[1]*self.rescale+480)
            self.button_city[settlement].place_forget()

    def create_button_edges(self, road):
        self.button_road[road]=ttk.Button(self.map,text='road',command=lambda : self.place_road(road))
        self.button_road[road].place(in_=self.map, x=road.coordinates[0]*self.rescale+480, y=road.coordinates[1]*self.rescale+480)
        self.button_road[road].place_forget()


    def show_buttons_robber(self):
        for region in self.button_robber:
            self.button_robber[region].place(in_=self.map, x=region.coordinates[0]*self.rescale+480, y=region.coordinates[1]*self.rescale+480)
            self.button_robber[region].state(['!disabled'])
        self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
    


    def show_buttons_settlements(self, round=1):
        self.current_player=self.game.current_player
        if round==0:
            self.board.round0_available_settlements(self.game.players[self.current_player])
            for settlement in self.game.players[self.current_player].available_settlements:#change to available button settlements
                self.button_settlement[settlement].place(in_=self.map, x=settlement.coordinates[0]*self.rescale+480, y=settlement.coordinates[1]*self.rescale+480)
            self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
        else:
            self.board.round_available_settlements(self.game.players[self.current_player])
            if (self.game.players[self.current_player].resources['brick']>0 and self.game.players[self.current_player].resources['grain']>0 
            and self.game.players[self.current_player].resources['wool']>0 and self.game.players[self.current_player].resources['lumber']>0):   
                if len(self.game.players[self.current_player].available_settlements)!=0:
                    for settlement in self.game.players[self.current_player].available_settlements:#change to available button settlements
                        self.button_settlement[settlement].place(in_=self.map, x=settlement.coordinates[0]*self.rescale+480, y=settlement.coordinates[1]*self.rescale+480)
                    self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                else:
                    self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                    self.enable_button(self.end_action_button)
            else:
                self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                self.enable_button(self.end_action_button)
        self.var.set(0)
        self.end_action_button.wait_variable(self.var)
        

    def show_buttons_roads(self, round=1):
        self.current_player=self.game.current_player
        if round==0:
            self.board.round0_available_roads(self.game.players[self.current_player])
            for road in self.game.players[self.current_player].available_roads:
                    self.button_road[road].place(in_=self.map, x=road.coordinates[0]*self.rescale+480, y=road.coordinates[1]*self.rescale+480)
            self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
        else:
            self.board.round_available_roads(self.game.players[self.current_player])
            if self.game.players[self.current_player].resources['brick']>0 and self.game.players[self.current_player].resources['lumber']>0:
                if len(self.game.players[self.current_player].available_roads)!=0:
                    for road in self.game.players[self.current_player].available_roads:
                        self.button_road[road].place(in_=self.map, x=road.coordinates[0]*self.rescale+480, y=road.coordinates[1]*self.rescale+480)
                    self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                else:
                    self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                    self.enable_button(self.end_action_button)
            else:
                self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
                self.enable_button(self.end_action_button)
        
        self.var.set(0)
        self.end_action_button.wait_variable(self.var)


    def show_buttons_up_settlements(self):
        self.current_player=self.game.current_player
        if self.game.players[self.current_player].resources['ore']>2 and self.game.players[self.current_player].resources['grain']>1:
            for settlement in self.game.players[self.current_player].settlements:
                    if settlement.multiplier==1:
                        self.button_city[settlement].place(in_=self.map, x=settlement.coordinates[0]*self.rescale+480, y=settlement.coordinates[1]*self.rescale+480)
            self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button, self.end_action_button)
        else:
            self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
            self.enable_button(self.end_action_button)
        self.var.set(0)
        self.end_action_button.wait_variable(self.var)
    
    def disable_button(self, *args):
        for button in args:
            button.state(['disabled'])

    def enable_button(self, *args):
        for button in args:
            button.state(['!disabled'])    
    
    def end_action(self):
        self.var.set(1)
        self.disable_button(self.end_action_button)
        if self.round==0:
            self.enable_button(self.end_turn_button)
        else:
            self.enable_button(self.end_turn_button, self.buy_settlement, self.buy_road, self.up_settlement)
        

    def end_turn(self):
        self.var_end_turn.set(1)
        self.disable_button(self.end_turn_button)

    def action_place_robber(self):
        self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button, self.end_action_button)
        self.enable_button(self.place_robber)
        self.var.set(0)
        self.end_action_button.wait_variable(self.var)



    def other_actions(self):
        self.round=1
        self.enable_button(self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button)
        self.var_end_turn.set(0)
        self.end_turn_button.wait_variable(self.var_end_turn)
        


    def update_player_label(self):
        self.current_player_label.config(text=f'CURRENT PLAYER : {self.game.current_player}', foreground=self.RGB_players[self.game.current_player])

    def update_player_resources(self):
        self.player_resources.config(text=f'{self.game.players[self.game.current_player].resources}')

    def update_dice_roll(self, dice):
        self.dice.config(text=f'dice roll: {dice}')


    def display_winner(self, winner):
        self_winner=Label(self.panel, text=f'THE WINNER IS {winner}', background = "wheat")
        self.dice.place(in_=self.panel, x=200, y=300)

    def setup_panel(self):
        self.place_robber=ttk.Button(self.panel, text='MOVE ROBBER',command=self.show_buttons_robber )
        self.place_robber.place(in_=self.panel, x=50, y=200)
        self.buy_road=ttk.Button(self.panel, text='BUY ROAD',command=self.show_buttons_roads)
        self.buy_road.place(in_=self.panel, x=50, y=50)
        self.buy_settlement=ttk.Button(self.panel, text='BUY SETTLEMENT',command=self.show_buttons_settlements )
        self.buy_settlement.place(in_=self.panel, x=50, y=100)
        self.up_settlement=ttk.Button(self.panel, text='UPGRADE SETTLEMENT',command=self.show_buttons_up_settlements )
        self.up_settlement.place(in_=self.panel, x=50, y=150)
        self.end_turn_button=ttk.Button(self.panel, text='END TURN',command=self.end_turn )
        self.end_turn_button.place(in_=self.panel, x=50, y=250)
        self.end_action_button=ttk.Button(self.panel, text='END ACTION',command=self.end_action )
        self.end_action_button.place(in_=self.panel, x=50, y=300)
        self.disable_button(self.place_robber, self.buy_settlement, self.buy_road, self.up_settlement, self.end_turn_button, self.end_action_button)
        self.current_player_label=ttk.Label(self.panel, text=f'CURRENT PLAYER : {self.game.current_player}', background = "wheat")
        self.current_player_label.place(in_=self.panel, x=50, y=350)

        self.player_label=ttk.Label(self.panel, text='current player resources:', background = "wheat")
        self.player_label.place(in_=self.panel, x=50, y=500)
        self.player_resources=ttk.Label(self.panel, text=f'', background = "wheat")
        self.player_resources.place(in_=self.panel, x=50, y=520)

        self.dice=ttk.Label(self.panel, text='dice roll:', background = "wheat")
        self.dice.place(in_=self.panel, x=200, y=200)


    def setup_map(self):
        for region in self.board.regions:
            self.place_image_region(region)
        
        for settlement in self.board.settlements:
            self.create_button_vertices(settlement, city=0)
            self.create_button_vertices(settlement, city=1)
        for road in self.board.roads:
            self.create_button_edges(road)



        self.robber_image=self.map.create_image(self.board.robber.region.coordinates[0]*self.rescale, self.board.robber.region.coordinates[1]*self.rescale, image = self.robber)
        

    def rowcol(self, event):
        self.frame["text"] = f"{self.map.canvasx(event.x)} | {self.map.canvasy(event.y)}"




    def build_GUI(self):
       # self.map.bind("<Motion>", self.rowcol)
        self.master.mainloop()

In [4]:

window=GUI()
window.build_GUI()